In [13]:
import numpy as np
import pandas as pd

In [14]:
urls = ['https://fbref.com/en/comps/Big5/2022-2023/shooting/players/2022-2023-Big-5-European-Leagues-Stats'
       'https://fbref.com/en/comps/Big5/2021-2022/shooting/players/2021-2022-Big-5-European-Leagues-Stats',
       'https://fbref.com/en/comps/Big5/2020-2021/shooting/players/2020-2021-Big-5-European-Leagues-Stats',
       'https://fbref.com/en/comps/Big5/2019-2020/shooting/players/2019-2020-Big-5-European-Leagues-Stats',
       'https://fbref.com/en/comps/Big5/2018-2019/shooting/players/2018-2019-Big-5-European-Leagues-Stats',
       'https://fbref.com/en/comps/Big5/2017-2018/shooting/players/2017-2018-Big-5-European-Leagues-Stats']

In [ ]:
xG_df = pd.DataFrame()

for url in urls:
    df = pd.read_html(url)[0]
    df.columns = [' '.join(col).strip() for col in df.columns]
    df = df.reset_index(drop=True)
    
    #data manipultation
    new_columns = []
    for col in df.columns:
      if 'level_0' in col:
          new_col = col.split()[-1]  # takes the last name
      else:
          new_col = col
      new_columns.append(new_col)

    # rename columns
    df.columns = new_columns
    df = df.fillna(0)
    
    df.columns = df.columns.str.replace('Standard ', '')
    df.columns = df.columns.str.replace('Expected ', '')

    df['Year'] = url[url.find("players/")+8 : url.find("players/")+17]
    df['Age'] = df['Age'].str[:2]
    df['Position'] = df['Pos'].str[:2]
    df['Position_2'] = df['Pos'].str[3:]
    df['Nation'] = df['Nation'].str.split(' ').str.get(1)
    df['League'] = df['Comp'].str.split(' ').str.get(1)
    df['League_'] = df['Comp'].str.split(' ').str.get(2)
    df['League'] = df['League'] + ' ' + df['League_']
    df = df.drop(columns=['Rk', 'Comp', 'Rk', 'Pos','Matches', 'League_'])
    df['Position'] = df['Position'].replace({'MF': 'Midfielder', 
                                             'DF': 'Defender', 
                                             'FW': 'Forward', 
                                             'GK': 'Goalkeeper'})
    df['Position_2'] = df['Position_2'].replace({'MF': 'Midfielder', 
                                                 'DF': 'Defender',
                                                 'FW': 'Forward', 
                                                 'GK': 'Goalkeeper'})
    df['League'] = df['League'].fillna('Bundesliga')
    
    xG_df = xG_df.append(df)

In [ ]:
xG_df = xG_df[xG_df['Age'] != 'Ag']
xG_df = xG_df[xG_df['Position'] != 'Goalkeeper']

In [ ]:
xG_df[['Age', 'Born', '90s', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 
       'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG']] = xG_df[['Age', 'Born', '90s', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90','SoT/90',
                                                             'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG', 
                                                             'npxG/Sh', 'G-xG', 'np:G-xG']].apply(pd.to_numeric) 

In [ ]:
league_games = {'League': ['Premier League', 'Bundesliga', 'La Liga', 'Serie A', 'Ligue 1'], 'Games': [38, 36, 38, 38, 38]}

league_games_df = pd.DataFrame(league_games)

In [ ]:
xG_df = xG_df.join(league_games_df.set_index('League'), on='League')


In [ ]:
xG_df['Cutoff']=0.3*xG_df['Games']
xG_df['Qualifying'] = np.where(xG_df['90s'] >= xG_df['Cutoff'], 1, 0)

In [ ]:
xG_df[xG_df['Qualifying']==1]

In [ ]:
xG_df.sort_values(by=['np:G-xG'])